<a href="https://colab.research.google.com/github/adnanahmed60222-cmd/nooha-asp/blob/main/decision_tree_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
df = pd.read_csv('Train.csv')
df.head(20)

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A
5,461319,Male,Yes,56,No,Artist,0.0,Average,2.0,Cat_6,C
6,460156,Male,No,32,Yes,Healthcare,1.0,Low,3.0,Cat_6,C
7,464347,Female,No,33,Yes,Healthcare,1.0,Low,3.0,Cat_6,D
8,465015,Female,Yes,61,Yes,Engineer,0.0,Low,3.0,Cat_7,D
9,465176,Female,Yes,55,Yes,Artist,1.0,Average,4.0,Cat_6,C


In [14]:
df['Segmentation'].value_counts()

,count
Segmentation,
D,2268
A,1972
C,1970
B,1858


In [4]:
df = df.drop('ID',axis = 1)



In [5]:
df.head(5)

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [6]:
#data cleaning part
df['Work_Experience'] = df['Work_Experience'].fillna(df['Work_Experience'].mean())
df['Family_Size'] = df['Family_Size'].fillna(df['Family_Size'].mean())
df['Ever_Married'] = df['Ever_Married'].fillna(df['Ever_Married'].mode()[0])
df['Graduated'] = df['Graduated'].fillna(df['Graduated'].mode()[0])
df['Profession'] = df['Profession'].fillna(df['Profession'].mode()[0])
df['Var_1'] = df['Var_1'].fillna(df['Var_1'].mode()[0])


In [7]:
#label encoding
import pandas as pd

categorical_cols = [col for col in ['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score', 'Var_1'] if col in df.columns]

df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

#  Convert only boolean dummy columns to int
bool_cols = df.select_dtypes(include=['bool']).columns
df[bool_cols] = df[bool_cols].astype(int)





In [8]:
#split the data
from sklearn.model_selection import train_test_split
X = df.drop('Segmentation', axis=1)
y = df['Segmentation']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
num_cols = ['Age', 'Work_Experience', 'Family_Size']
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
params = {
    'n_estimators': [100, 200],
    'max_depth': [7, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

grid = GridSearchCV(RandomForestClassifier(random_state=42),
                    param_grid=params,
                    cv=15,
                    scoring='accuracy',
                    n_jobs=-1)

grid.fit(X_train, y_train)

print("Best Parameters:", grid.best_params_)
print("Best CV Accuracy:", grid.best_score_)

Best Parameters: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}
Best CV Accuracy: 0.5323202964802378


In [17]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
cv = StratifiedKFold(n_splits=25)
scores = cross_val_score(grid.best_estimator_, X_train, y_train, cv=cv, scoring='accuracy')
print("Mean CV Accuracy:", scores.mean())


y_pred = grid.best_estimator_.predict(X_test)

print(f"Test Accuracy: {accuracy_score(y_test, y_pred)}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Mean CV Accuracy: 0.5337250737463127
Test Accuracy: 0.5324246179264767

Classification Report:
               precision    recall  f1-score   support

           A       0.46      0.47      0.46       597
           B       0.44      0.30      0.36       552
           C       0.54      0.58      0.56       580
           D       0.63      0.74      0.68       692

    accuracy                           0.53      2421
   macro avg       0.52      0.52      0.51      2421
weighted avg       0.52      0.53      0.52      2421

